#### 파일 1 개만 제출 가능하여 여러 개의 주피터 작업 본을 합친 양식이라 중간 전개가 약간 어색 할 수 있습니다.  
#### 끝에는 py 파일도 코드를 넣었으니 참고 부탁 드립니다. 

In [ ]:
# 가로로 넓게 쓰기
from IPython.core.display import display, HTML  
display(HTML(""))

In [ ]:
import time 
import datetime
from datetime import datetime

import os
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
plt.style.use('seaborn-whitegrid')

import pickle
import joblib

# ML Library
import pandas_profiling 
from tqdm.auto import tqdm

from sklearn import tree
from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import lightgbm as lgb
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV, GroupKFold

from sklearn import metrics
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, precision_score, auc, precision_recall_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import mean_absolute_error

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek # 오버 샘플링 성능 비교용 
from imblearn.under_sampling import TomekLinks

from adamp import AdamP
from tensorflow_addons.optimizers import AdamW

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import GRU, Dropout, Dense, Conv1D
from tensorflow.keras import models

# importing plotly and cufflinks in offline mode
# import cufflinks as cf
# import plotly.offline as pyo
# cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

# pyo.init_notebook_mode()

# TPOT : Automatic ML library that automatically enables feature selection and Hyperparameter learning based on multiple machine learning models 
from tpot import TPOTClassifier

import warnings
warnings.filterwarnings('ignore')

print("Setup Complete")

In [ ]:
path = "/home/kym/ML/input/kamp/대회 과제"
os.chdir(path)

df = pd.read_csv('melting_tank.csv', encoding='cp949')

In [ ]:
print(df.shape)

In [ ]:
df.info()

In [ ]:
# df.profile_report()
# profile = df.profile_report(title='Competition_Pandas Profiling Report')
# profile.to_file(output_file="Competition data profiling.html")

In [ ]:
df['MELT_TEMP'] = (df['MELT_TEMP'] / 10) # 용해온도, 교반속도 데이터는 소수점 1 자리 생략 
df['MOTORSPEED'] = (df['MOTORSPEED'] / 10)

result_mapping = {
    "OK": 1,
    "NG": 0
}

df.loc[:, "TAG"] = df.TAG.map(result_mapping)

In [ ]:
# Heatmap

fig, ax = plt.subplots(1, 1, figsize=(20, 20))
df_corr = df.corr()

mask = np.zeros_like(df_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(df_corr, ax=ax,
           square=True, center=0, linewidth=1,
           cmap=sns.diverging_palette(240, 10, as_cmap=True),
           cbar_kws={'shrink': .82},
           mask=mask,
           annot=True,
           annot_kws={'size':7}
           )
ax.set_title(f'Correlation', loc='left', fontweight='bold')
plt.show()

In [ ]:
# # 수분 함유량 (INSP) 가 3.21 ~ 3.23 은 불량이 극히 적다. (높을수록 정상이 많음.)
# df[['INSP', 'TAG']].groupby(['INSP'], as_index=True).mean().iplot(kind='bar')

In [ ]:
df['STD_DT'] = df['STD_DT'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %H:%M', errors='coerce'))
df.set_index('STD_DT', inplace=True)

In [ ]:
df.describe()

In [ ]:
for i in df.columns:
    print("=============================================")
    sns.distplot(df[i])
    plt.show()
    print()

In [ ]:
# 용량 문제로 주석 처리 
# import plotly.express as px

# fig = px.scatter(df, x='MOTORSPEED', y='INSP', marginal_y='histogram', marginal_x='histogram')
# fig.show()
# fig = px.scatter(df, x='MELT_WEIGHT', y='INSP', marginal_y='histogram', marginal_x='histogram')
# fig.show()
# fig = px.scatter(df, x='MELT_TEMP', y='INSP', marginal_y='histogram', marginal_x='histogram')
# fig.show()
# fig = px.scatter(df, x='INSP', y='TAG', marginal_y='histogram', marginal_x='histogram')
# fig.show()

In [ ]:
# 중량의 경우 너무 과한 값은 삭제 
sns.boxplot(x='MELT_WEIGHT', orient = "v", data=df)
plt.show()

In [ ]:
# 직접 관측 어려운 feature 는 제외 
df.drop(['INSP'], axis=1, inplace=True)

In [ ]:
df[['MELT_WEIGHT', 'TAG']].plot(kind='line', figsize=(20,6), linewidth=2, fontsize=20, colormap='winter')
plt.xlabel('time', fontsize=15)
plt.ylabel('MinMax label', fontsize=15)
plt.show()

In [ ]:
# 질량 전 처리 : 조건은 직전 값과 100이상 차이 나는 경우 시계열상 다다음 스텝(30스텝까지)의 무게와 비교해서 증감 비율 맞춰서 보정하는 형태
Dlength = df.shape[0]
s_step = 30

for k in df.NUM:
    if k >= 1:
        if abs(df.MELT_WEIGHT[k] - df.MELT_WEIGHT[k-1]) > 100:
            
            for s in range(s_step):
                if abs(df.MELT_WEIGHT[k-1] - df.MELT_WEIGHT[k+s]) < 500: 
                    step = round(abs(df.MELT_WEIGHT[k-1] - df.MELT_WEIGHT[k+s]) / (s+1))
                    
                    if df.MELT_WEIGHT[k-1] > df.MELT_WEIGHT[k+s]:
                        df.MELT_WEIGHT[k] = df.MELT_WEIGHT[k-1] - step
                    
                    else:
                        df.MELT_WEIGHT[k] = df.MELT_WEIGHT[k-1] + step
                    break

df.to_csv('/home/kym/ML/input/kamp/대회 과제/melting_tank_mod.csv')

In [ ]:
path = "/home/kym/ML/input/kamp/대회 과제"
os.chdir(path)

df = pd.read_csv('melting_tank_mod.csv', encoding='cp949')
df.head()

In [ ]:
# 데이터 수정 이 후 
df[['MELT_WEIGHT', 'TAG']].plot(kind='line', figsize=(20,6), linewidth=2, fontsize=20, colormap='winter')
plt.xlabel('time', fontsize=15)
plt.ylabel('MinMax label', fontsize=15)
plt.show()

In [ ]:
df['STD_DT'] = df['STD_DT'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df.info()

온도와 모터 스피드 간 높은 상관성

In [ ]:
# 피처별 점수
from functools import partial
from scipy import optimize

class OptimizeAUC:
    def __init__(self):
        self.coef_ = 0
        
    def _auc(self, coef, X, y):
        '''
        :params coef : weight list
        '''
        x_coef = X * coef
        predictions = np.sum(x_coef, axis=1)
        auc_score = metrics.roc_auc_score(y, predictions)
        
        return -1.0 * auc_score
    
    def fit(self, X, y):
        loss_partial = partial(self._auc, X=X, y=y)
        initial_coef = np.random.dirichlet(np.ones(X.shape[1]), size=1)
        self.coef_ = optimize.fmin(loss_partial, initial_coef, disp=True)
        
    def predict(self, X):
        x_coef = X * self.coef_
        predictions = np.sum(x_coef, axis=1)
        
        return predictions

In [ ]:
X = df.drop(['STD_DT', 'NUM', 'TAG'], axis=1)
y = df['TAG']
X.head()

In [ ]:
 랜덤 포레스트, light GBM, XGBoost, DL, KNN => ENSEMBLE

x_train, x_test, y_train, y_test = model_selection.train_test_split(
    X,
    y,
    test_size=0.5,
    stratify=y,
    shuffle=True,
    random_state=42
)

scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

lgb = lgb.LGBMClassifier(num_leaves=31, 
                         objective='binary',
                         num_iterations=200,
                         max_depth=5
                        )
rf = ensemble.RandomForestClassifier(random_state=42)
knn = KNeighborsClassifier(n_jobs=-1)
xgb = xgb.XGBClassifier(random_state=42,
                       booster='gbtree',
                       n_jobs=-1,
                       eval_metric='error',
                       learning_rate=0.1,
                       n_estimators=200,
                       max_depth=5,
                       reg_lambda=2
                       )

lgb.fit(x_train_scaled, y_train)
rf.fit(x_train_scaled, y_train)
knn.fit(x_train_scaled, y_train)
xgb.fit(x_train_scaled, y_train)

pred_lgb = lgb.predict_proba(x_test_scaled)[:, 1]
pred_rf = rf.predict_proba(x_test_scaled)[:, 1]
pred_knn = knn.predict_proba(x_test_scaled)[:, 1]
pred_xgb = xgb.predict_proba(x_test_scaled)[:, 1]
avg_pred = (pred_xgb + pred_rf + pred_knn + pred_xgb) / 4

test_preds = np.column_stack((
            pred_lgb,
            pred_rf,
            pred_knn,
            pred_xgb,
            avg_pred
))

aucs_test = []

for i in range(test_preds.shape[1]):
    auc = metrics.roc_auc_score(y_test, test_preds[:, i])
    aucs_test.append(auc)
    
print(f"테스트 셋: LightGBM AUC = {aucs_test[0]}")
print(f"테스트 셋: RandomForest AUC = {aucs_test[1]}")
print(f"테스트 셋: KNN AUC = {aucs_test[2]}")
print(f"테스트 셋: XGBoost AUC = {aucs_test[3]}")
print(f"테스트 셋: Average Pred AUC = {aucs_test[4]}")

In [ ]:
lgb.fit(x_test_scaled, y_test)
rf.fit(x_test_scaled, y_test)
knn.fit(x_test_scaled, y_test)
xgb.fit(x_test_scaled, y_test)

pred_lgb = lgb.predict_proba(x_train_scaled)[:, 1]
pred_rf = rf.predict_proba(x_train_scaled)[:, 1]
pred_knn = knn.predict_proba(x_train_scaled)[:, 1]
pred_xgb = xgb.predict_proba(x_train_scaled)[:, 1]
avg_pred = (pred_xgb + pred_rf + pred_knn + pred_xgb) / 4

train_preds = np.column_stack((
            pred_lgb,
            pred_rf,
            pred_knn,
            pred_xgb,
            avg_pred
))

aucs_train = []

for i in range(train_preds.shape[1]):
    auc = metrics.roc_auc_score(y_train, train_preds[:, i])
    aucs_train.append(auc)
    
print(f"테스트 셋: LightGBM AUC = {aucs_train[0]}")
print(f"테스트 셋: RandomForest AUC = {aucs_train[1]}")
print(f"테스트 셋: KNN AUC = {aucs_train[2]}")
print(f"테스트 셋: XGBoost AUC = {aucs_train[3]}")
print(f"테스트 셋: Average Pred AUC = {aucs_train[4]}")

In [ ]:
opt = OptimizeAUC()
opt.fit(train_preds[:, :-1], y_train)
opt_preds_test = opt.predict(test_preds[:, :-1])
auc = metrics.roc_auc_score(y_test, opt_preds_test)

print(f"Optimized AUC, Test set = {auc}")
print(f"Coefficients = {opt.coef_}")

opt = OptimizeAUC()
opt.fit(test_preds[:, :-1], y_test)
opt_preds_train = opt.predict(train_preds[:, :-1])
auc = metrics.roc_auc_score(y_train, opt_preds_train)

print(f"Optimized AUC, Train set = {auc}")
print(f"Coefficients = {opt.coef_}")  

* 피처 1 개 (온도) : 0.774 (optimized auc 0.78)  
* 피처 1 개 (모터 속도) : 0.73 (optimized auc 0.736)  
* 피처 2 개 (온도, 모터 속도) : 0.77 (optimized auc 0.78)  
* 피처 3 개 (온도, 모터 속도, 중량): 0.79 (optimized auc 0.80)  
* 피처 4 개 (온도, 모터 속도, 중량, insp) : 0.79 (optimized auc 0.80)  

온도, 모터 회전 수까지만 학습에 사용 : Metric 뿐만 아니라 데이터 수집 측면에서 INSP 는 불리하고 중량은 영향도가 미미하기 때문임.

feature 확장 가능성 검토

In [ ]:
def generate_features(df):
    df.loc[:,  'year'] = df['STD_DT'].dt.year
    df.loc[:, 'weekofyear'] = df['STD_DT'].dt.weekofyear
    df.loc[:, 'month'] = df['STD_DT'].dt.month
    df.loc[:, 'dayofweek'] = df['STD_DT'].dt.dayofweek
    df.loc[:, 'weekend'] = (df['STD_DT'].dt.weekday >=5).astype(int)
#     df.loc[:, 'hour'] = df['STD_DT'].dt.hour
#     df.loc[:, 'min'] = df['STD_DT'].dt.mimute
    
    aggs = {}
#     aggs['month'] = ['nunique', 'mean']
#     aggs['weekofyear'] = ['mean']
    aggs['MELT_TEMP'] = ['max', 'min', 'mean']
    aggs['MOTORSPEED'] = ['max', 'min', 'mean']
    aggs['MELT_WEIGHT'] = ['max', 'min', 'mean']
#     aggs['NUM'] = ['size']
#     aggs['NUM'] = ['nunique']
    
    agg_df = df.groupby('NUM').agg(aggs)
    agg_df = agg_df.reset_index()
    
    return agg_df

In [ ]:
df1 = generate_features(df)
df1.head()

In [ ]:
df = pd.concat([df, df1], axis=1)
df.head()

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
np.var(df)

In [ ]:
# Heatmap

fig, ax = plt.subplots(1, 1, figsize=(20, 20))
df_corr = df.corr()

mask = np.zeros_like(df_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(df_corr, ax=ax,
           square=True, center=0, linewidth=1,
           cmap=sns.diverging_palette(240, 10, as_cmap=True),
           cbar_kws={'shrink': .82},
           mask=mask,
           annot=True,
           annot_kws={'size':7}
           )
ax.set_title(f'Correlation', loc='left', fontweight='bold')
plt.show()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
features = [
    f for f in df.columns if f not in ('STD_DT', 'NUM', 'TAG', 'year', 'weekofyear', 'month', 'dayofweek', 'weekend', ('NUM', ''))
]
df[features] = scaler.fit_transform(df[features])
X = df.drop(['STD_DT', 'NUM', ('NUM', ''), 'TAG'], axis=1)
y = df['TAG'] 

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
rf = ensemble.RandomForestClassifier(random_state=42)
rfe = RFE(
    estimator=rf,
    n_features_to_select=5)

rfe.fit(X, y)

X_transformed = rfe.transform(X)

In [ ]:
# melt_temp, weekofyear, dayofweek
X_transformed

In [ ]:
X

In [ ]:
def generate_features(df):
    df['STD_DT'] = df['STD_DT'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df.loc[:, 'weekofyear'] = df['STD_DT'].dt.weekofyear
    df.loc[:, 'dayofweek'] = df['STD_DT'].dt.dayofweek
    df.loc[:, 'weekend'] = (df['STD_DT'].dt.weekday >=5).astype(int)
#     df.loc[:, 'hour'] = df['STD_DT'].dt.hour
#     df.loc[:, 'min'] = df['STD_DT'].dt.mimute
    
    return df

In [ ]:
path = "/home/kym/ML/input/kamp/대회 과제"
os.chdir(path)

df = pd.read_csv('melting_tank_mod.csv', encoding='cp949')

df = generate_features(df)
feature = [
    f for f in df.columns if f in ("STD_DT", "MELT_TEMP", "MOTORSPEED", "MELT_WEIGHT", "weekofyear", "dayofweek", "weekend", "TAG")
]

df = df[feature]

df.to_csv('/home/kym/ML/input/kamp/대회 과제/melting_tank_final.csv', index=False)
df.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
scaler_features = [
    f for f in df.columns if f in ('MELT_TEMP', 'MOTORSPEED', 'MELT_WEIGHT')
]
df[scaler_features] = scaler.fit_transform(df[scaler_features])

In [ ]:
X = df.drop(['STD_DT', 'TAG'], axis=1)
y = pd.DataFrame(df['TAG'])

In [ ]:
# 1 정상, 0 불량 (가이드 북 기준, 본래는 유저가 관심 있는 것을 보통 1 로 사용)
print("정상 개수 : {} ({:.2f} %)".format(y.TAG.value_counts()[1], y.TAG.value_counts(normalize=True)[1]))
print("비 정상 개수: {} ({:.2f} %)".format(y.TAG.value_counts()[0], y.TAG.value_counts(normalize=True)[0])) 

In [ ]:
for col in X.columns:
    fig = plt.figure()
    ax = plt.axes()
    plt.title(col)
    x = np.linspace(0, X.shape[0])
    ax.plot(X[col])
    ax.plot(y['TAG'])

근무일과 같은 시간에 주기성을 띄기 보단 그냥 특정 시점에 집중적으로 불량이 몰려 있음.  
따라서, 평일, 주 차 등의 근무 연관성은 낮다고 봐야 함.  
(근무자가 불량이거나 투입 중량 시스템 오류 등 데이터에 명시되지 않은 다른 원인 추정 가능)  

불량 구간 확대 관찰

In [ ]:
# 불량 발생 지점 확대, 그림이 잘 보이게 하기 위해 첫 번째는 라벨 없이 데이터만 도시 
eventrow = 200000

minx = eventrow - 50
maxx = eventrow + 50

for col in X.columns:
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure()
    ax = plt.axes()
    plt.title(col)
    
    x = np.linspace(minx, maxx)
    ax.plot(X[col][minx : maxx])
#     ax.plot(y['TAG'][minx : maxx])

In [ ]:
# 불량 발생 지점 확대, 라벨과 비교 : 시간적인 feature (작업 시간, 출 퇴근이나 작업자 성향을 간접 파악) 들과 전혀 관계 없으며 , 
# 재료 질량과도 거의 상관 없음. (LInear 한 감소에서 주기적 라벨링 관찰) 
# 온도와 모터 스피드 변화에 따라 광범위한 일정 영역 대에서 집중 발생함을 알 수 있음. 

eventrow = 200000

minx = eventrow - 50
maxx = eventrow + 50

for col in X.columns:
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure()
    ax = plt.axes()
    plt.title(col)
    
    x = np.linspace(minx, maxx)
    ax.plot(X[col][minx : maxx])
    ax.plot(y['TAG'][minx : maxx])

정상 구간 확대 관찰

In [ ]:
# 불량 발생 지점 확대, 그림이 잘 보이게 하기 위해 첫 번째는 라벨 없이 데이터만 도시 
eventrow = 100000

minx = eventrow - 50
maxx = eventrow + 50

for col in X.columns:
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure()
    ax = plt.axes()
    plt.title(col)
    
    x = np.linspace(minx, maxx)
    ax.plot(X[col][minx : maxx])
#     ax.plot(y['TAG'][minx : maxx])

In [ ]:
# 정상 구간 지점 확대, 라벨과 비교 : 시간적인 feature (작업 시간, 출 퇴근이나 작업자 성향을 간접 파악) 들과 전혀 관계 없으며 , 
# 재료 질량과도 거의 상관 없음. (LInear 한 감소에서 주기적 라벨링 관찰) 
# 온도와 모터 스피드 변화에 따라 광범위한 일정 영역 대에서 집중 발생함을 알 수 있음. 

eventrow = 100000

minx = eventrow - 50
maxx = eventrow + 50

for col in X.columns:
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure()
    ax = plt.axes()
    plt.title(col)
    
    x = np.linspace(minx, maxx)
    ax.plot(X[col][minx : maxx])
    ax.plot(y['TAG'][minx : maxx])

In [ ]:
X['TAG'] = y['TAG']

normal = X[X['TAG'] == 1]
anomaly = X[X['TAG'] == 0]

print(normal.shape)
print(anomaly.shape)

In [ ]:
# 정상과 비 정상의 통계 정보 : 날짜 관련은 TAG 와 상관 없음을 알 수 있다. 
normal.describe()

In [ ]:
anomaly.describe()

In [ ]:
from plotly import graph_objs as go
from plotly.offline import iplot

In [ ]:
# GIT 업로드 용량 문제로 주석 처리 : 온도 - 모터 속도 - 중량 순으로 연관도를 보임. 

# features = ['MELT_TEMP', 'MOTORSPEED', 'MELT_WEIGHT']

# for col in features:
#     trace0 = go.Box(
#                     y=normal[col],
#                     name="normal state",
#                     marker=dict(
#                     color = 'rgb(12, 12, 140)',
#                     )
#     )
    
#     trace1 = go.Box(
#                     y=anomaly[col],
#                     name="anomaly state",
#                     marker=dict(
#                     color = 'rgb(12, 128, 128)',
#                     )
#     )
#     data = [trace0, trace1]
#     iplot(data)

모델링

In [ ]:
path = "/home/kym/ML/input/kamp/대회 과제"
os.chdir(path)
model_path = path

df = pd.read_csv('melting_tank_mod.csv', encoding='cp949')

feature = [
    f for f in df.columns if f not in ("NUM")
]

df = df[feature]
df['STD_DT'] = df['STD_DT'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %H:%M:%S', errors='coerce'))
df.set_index('STD_DT', inplace=True)

df.to_csv('/home/kym/ML/input/kamp/대회 과제/melting_tank_final.csv', index=False)
df.head()

In [ ]:
# 표준 편차가 큰 feature 인 재료의 질량은 관찰 그래프를 통해 극히 영향도가 미미한 것을 알았기에 제외하고, 탱크 온도와 모터 속도만으로 학습함. 
df.describe().T.style.bar(subset=['mean'], color='#205ff2').background_gradient(subset=['std'], cmap='Reds')

In [ ]:
# 전체 데이터 83 만여개를 모두 사용하여 학습할 경우 real time detection 이 전부 정상으로 나오는 문제가 발생함.
# 따라서, 정상 구간 부분을 버리고 OK / NG 가 모두 존재하는 부분만 학습을 진행함. 

start_point = 189000
end_point = 580000

df = df.iloc[start_point:end_point, :]
df.head()

In [ ]:
df.shape

In [ ]:
X = df.drop(['MELT_WEIGHT', 'TAG'], axis=1)
y = df['TAG']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'), random_state=42)
x_train_over, y_train_over = smoteto.fit_resample(x_train, y_train)

print("SMOTETomek 적용 전 학습용 피처 / 라벨 세트: ", x_train.shape, y_train.shape)
print("SMOTETomek 적용 후 학습용 피처 / 라벨 세트: ", x_train_over.shape, y_train_over.shape)
print("SMOTETomek 적용 후 라벨 값 분포: \n", pd.Series(y_train_over).value_counts())

In [ ]:
def process_data(df, label, window_size):
    feature_list = []
    label_list = []
    
    for i in range(len(df) - window_size):
        feature_list.append(np.array(df.iloc[i : i + window_size]))
        label_list.append(np.array(label.iloc[i + window_size]))
        
    return np.array(feature_list), np.array(label_list)

In [ ]:
# dataframe
x_train_over = pd.DataFrame(x_train_over, columns=['MELT_TEMP', 'MOTORSPEED'])
y_train_over = pd.DataFrame(y_train_over, columns=['TAG'])

In [ ]:
# window size = 10
train_feature, train_label = process_data(x_train_over, y_train_over, 10)
train_feature.shape, train_label.shape

In [ ]:
x_test = pd.DataFrame(x_test, columns=['MELT_TEMP', 'MOTORSPEED'])

In [ ]:
test_feature, test_label = process_data(x_test, y_test, 10)
test_feature.shape, test_label.shape

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.3)
x_train.shape, x_valid.shape

모델

In [ ]:
model = Sequential()
model.add(GRU(64,
              input_shape=(train_feature.shape[1], train_feature.shape[2]),
              activation='tanh',
              return_sequences=True)
         )
model.add(tf.keras.layers.Dropout(0.2))
model.add(GRU(16,
              activation='tanh',
              return_sequences=True)
         )
model.add(tf.keras.layers.Dropout(0.2))
model.add(GRU(4,
              activation='tanh',
              return_sequences=False)
         )
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
# Aactivation function 

# step = tf.Variable(0, trainable=False)
# schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
#     [10000, 15000], [1e-0, 1e-1, 1e-2])
# # lr and wd can be a function or a tensor
# lr = 1e-3 * schedule(step)
# wd = lambda: 1e-4 * schedule(step)

# AdamW

In [ ]:
# 현실 고려 시 FN (음성으로 예측했는데 실제로 양성인 것 = 정상 (1) 으로 예측했는데 실제로 불량 (0) 인 것) 을 중점적으로 개선해야 한다. 

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adamax(), metrics=[tf.keras.metrics.AUC(curve='ROC')])
early_stop = EarlyStopping(monitor='val_loss', patience=5, min_delta = 0.003)
filename = os.path.join(model_path, 'melting_tank_pretrained_model.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=2, save_best_only=True, mode='auto')

In [ ]:
history = model.fit(x_train, y_train,
                   epochs=20,
                   batch_size=64,
                   validation_data=(x_valid, y_valid),
                   callbacks=[early_stop, checkpoint])

In [ ]:
scaler_filename = 'mx_rscaler.pkl'
joblib.dump(scaler, scaler_filename)

In [ ]:
pred = model.predict(test_feature)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

In [ ]:
pred_df = pd.DataFrame(pred, columns=['TAG'])
pred_df['TAG'] = pred_df['TAG'].apply(lambda x: 1 if x >= 0.5 else 0)

In [ ]:
# 1 정상, 0 불량
pred_df['TAG'].value_counts()

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # print("Normalized confusion matrix")
    else:
        1 # print('Confusion matrix, without normalization')

    # print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# 연속된 공정은 실제 불량을 정상으로 예측하는 FN 을 줄여야 함. 실제 정상을 불량으로 예측 시 설비를 정지시키는 수준이지만, 실제 불량을 정상 예측할 경우 다음 공정까지
# 투입되는 모든 비용을 망치게 된다. 따라서 재현율 (recall) 중점으로 봐야 함. 

classify = confusion_matrix(test_label, pred_df)

plt.figure()
plot_confusion_matrix(classify, classes=[0, 1], title='Confusion matrix')
plt.show()

In [ ]:
print(classify)

In [ ]:
p = precision_score(test_label, pred_df)
print("precision: %0.4f" %p)
r = recall_score(test_label, pred_df)
print("recall: %0.4f" %r)
f1 = f1_score(test_label, pred_df)
print("f1-score: %0.4f" %f1)
acc = accuracy_score(test_label, pred_df)
print("accuracy: %0.4f" %acc)

In [ ]:
precision, recall, thresholds = precision_recall_curve(test_label, pred_df)

print("f1-score: %0.4f" %f1)
plt.plot([0, 1], [0, 0], linestyle='--')
plt.plot(recall, precision, marker='.')
plt.show()

### realtime.py 파이썬 파일  
1) mx_rscaler.pkl  
2) melting_tank_stream.csv (다운로드한 제조 데이터셋 원본과 같음)  
3) h5 pretrained model  

을 이용해 구동함.  

**결과는 PPT 자료의 캡처와 동영상 참조**   

In [ ]:
import time  
import pandas as pd 
import streamlit as st 
import joblib
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from keras.models import load_model



st.set_page_config(
    page_title="Dissolution Tank Condition",
    layout="wide",
)


# Data Prepare
FilePath = "/home/kym/ML/input/kamp/대회 과제/Streamlit-main/melting_tank_stream.csv"
scaler_call = joblib.load("/home/kym/ML/input/kamp/대회 과제/Streamlit-main/mx_rscaler.pkl") 
model_call = load_model('/home/kym/ML/input/kamp/대회 과제/Streamlit-main/melting_tank_pretrained_model.h5')

@st.experimental_memo
def get_data() -> pd.DataFrame:
    return pd.read_csv(FilePath)

df = get_data()
df_Length = df.shape[0]




# dashboard title

st.title("Dissolution Tank Condition")



placeholder_1 = st.empty()
placeholder_2 = st.empty()
# near real-time simulation
Pie_Value = [0, 0]
for seconds in range(df_Length):
    
    ndf = df.iloc[seconds:seconds+30]
    result_mapping = {
        "OK":1,
        "NG":0
    }
    ndf.loc[:,"TAG"]=ndf.TAG.map(result_mapping)
    
    
    
    with placeholder_1.container():
        Condition_Record, Condition_Est, Dummy_Area = st.columns((1,1,1))
        with Condition_Record:
            st.markdown("## Recorded TAG")
            Tag_Value=ndf["TAG"][seconds+29]
            if Tag_Value==1:
                st.success("OK")
            else:
                st.error("NG")
        with Condition_Est:
            
            st.markdown("## Estimated Condition")
            ndf.drop(['MELT_WEIGHT', 'TAG'], axis=1)
            
            new_x_df = pd.concat([ndf["MELT_TEMP"][20:],ndf["MOTORSPEED"][20:]] ,axis=1)
            new_x_df_scale = scaler_call.transform(new_x_df)  
            new_x_df_scale = new_x_df_scale.reshape(1,10,2)
            Tag_Est = model_call.predict(new_x_df_scale) 
            
            
            if Tag_Est > 0.5595:
                st.success("OK")
                Pie_Value[0] = Pie_Value[0] + 1
            else:
                st.error("NG")
                Pie_Value[1] = Pie_Value[1] + 1 
            
        
    with placeholder_2.container():
        
        Tank_Graph, Fail_Ratio, Tank_Data = st.columns((2,1,1))
        with Tank_Graph:
            st.markdown("## Tank Condition")
            fig = make_subplots(rows=4, cols=1,
                                subplot_titles=("Melting Temperature", "Motor Speed", "Melt Weight", "INSP"))
            fig.update_layout(margin=dict(l = 20,
                                         r=20,
                                         b=50,
                                         t=20,
                                         pad = 4))
            fig.add_trace(go.Scatter(x=ndf.NUM, y=ndf.MELT_TEMP,
                                        mode = "lines",
                                        name = 'Temp.'),
                                        row=1, col=1)
            fig.add_trace(go.Scatter(x=ndf.NUM, y=ndf.MOTORSPEED,
                                        mode = "lines",
                                        name = 'Motor Speed'),
                                        row=2, col=1)
            fig.add_trace(go.Scatter(x=ndf.NUM, y=ndf.MELT_WEIGHT,
                                        mode = "lines",
                                        name = 'WEIGHT'),
                                        row=3, col=1)
            fig.add_trace(go.Scatter(x=ndf.NUM, y=ndf.INSP,
                                        mode = "lines",
                                        name = 'INSP'),
                                        row=4, col=1)
            
            
            fig.add_annotation(x=ndf.NUM[seconds+29], y=ndf.MELT_TEMP[seconds+29],
                                text = 'CV',
                                showarrow = True,
                                arrowhead= 1,
                                row=1, col=1)
            fig.add_annotation(x=ndf.NUM[seconds+29], y=ndf.MOTORSPEED[seconds+29],
                                text = 'CV',
                                showarrow = True,
                                arrowhead= 1,
                                row=2, col=1)
            fig.add_annotation(x=ndf.NUM[seconds+29], y=ndf.MELT_WEIGHT[seconds+29],
                                text = 'CV',
                                showarrow = True,
                                arrowhead= 1,
                                row=3, col=1)
            fig.add_annotation(x=ndf.NUM[seconds+29], y=ndf.INSP[seconds+29],
                                text = 'CV',
                                showarrow = True,
                                arrowhead= 1,
                                row=4, col=1)
            st.plotly_chart(fig, use_container_width= True)
            
        with Fail_Ratio:
            st.markdown("## Estimated NG Ratio")
            labels = ['OK', 'NG']
            
            fig = go.Figure(data =[go.Pie(labels = labels, values= Pie_Value, hole=0.3)])
            st.plotly_chart(fig, use_container_width= True)
            
                
        with Tank_Data:
            st.markdown("## Recorded Data View")
            view_NDF = ndf.drop(['STD_DT', 'NUM'], axis=1)
            result_mapping = {
                1: "OK",
                0: "NG"
                }
            view_NDF.loc[:,"TAG"]=view_NDF.TAG.map(result_mapping)
            st.dataframe(view_NDF[20:])

        

        
        time.sleep(1)